In [0]:
import pytz
from datetime import datetime, timedelta

In [0]:
# Define the base path where the subfolders are located
departures_base_path = '/mnt/raw/departures/'
arrivals_base_path = '/mnt/raw/arrivals/'

# Use a wildcard (*) to read all Parquet files in all subfolders
df_departures = spark.read.parquet(f'{departures_base_path}*/')
df_arrivals = spark.read.parquet(f'{arrivals_base_path}*/')


departure_row = df_departures.count()
arrivals_row = df_arrivals.count()

print(departure_row)
print(arrivals_row)

72
30


In [0]:
df_departures_sorted = df_departures.orderBy("actual_departure_myt", ascending=True)
df_arrivals_sorted = df_arrivals.orderBy("actual_arrival_myt", ascending=True)

In [0]:
df_arrivals_sorted = df_arrivals.orderBy("actual_arrival_myt", ascending=True)

In [0]:
folder_path_arrivals = f'/mnt/compiled/arrivals/'
folder_path_departures = f'/mnt/compiled/departures/'

# Coalesce the DataFrame to a single partition
df_arrivals_coalesced = df_arrivals_sorted.coalesce(1)
df_departures_coalesced = df_departures_sorted.coalesce(1)

# Write the coalesced DataFrame to a Parquet file in the specified folder
df_arrivals_coalesced.write.mode('overwrite').parquet(folder_path_arrivals)
df_departures_coalesced.write.mode('overwrite').parquet(folder_path_departures)

# List the files in the directory after writing the Parquet file
files_arrivals = dbutils.fs.ls(folder_path_arrivals)
files_departures = dbutils.fs.ls(folder_path_departures)

# Correct the file path for renaming
corrected_file_path_arrivals = f"{folder_path_arrivals}arrivals_compiled.parquet"
corrected_file_path_departures = f"{folder_path_departures}departures_compiled.parquet"

# Find the part file and rename it (for arrivals)
for file in files_arrivals:
    if file.name.startswith("part-"):
        # Move (rename) the part file to the desired file name
        dbutils.fs.mv(file.path, corrected_file_path_arrivals)
    elif file.name.startswith("_SUCCESS") or file.name.startswith("_committed") or file.name.startswith("_started"):
        # Remove unwanted system files like _SUCCESS, _committed, _started
        dbutils.fs.rm(file.path)

# Find the part file and rename it (for departures)
for file in files_departures:
    if file.name.startswith("part-"):
        # Move (rename) the part file to the desired file name
        dbutils.fs.mv(file.path, corrected_file_path_departures)
    elif file.name.startswith("_SUCCESS") or file.name.startswith("_committed") or file.name.startswith("_started"):
        # Remove unwanted system files like _SUCCESS, _committed, _started
        dbutils.fs.rm(file.path)
